# Alpaca-py options trading basic

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/alpacahq/alpaca-py/blob/master/examples/options-trading-basic.ipynb)

- This notebook shows how to use alpaca-py with options trading API endpoints
- Please use ``paper account``. Please ``DO NOT`` use this notebook with live account. In this notebook, we place orders for options as an example.

In [15]:
# Please change the following to your own PAPER api key and secret
# You can get them from https://alpaca.markets/

# api_key = "<YOUR PAPER API KEY>"
# secret_key = "<YOUR PAPER API SECRET>"

api_key = "PKIZIAJPNX5J76APEZC2"
secret_key = "weSQCYiGHACg5WFwAW2SCwWZJTcuGUXux8SJQkqV"

#### We use paper environment for this example ####
paper = True # Please do not modify this. This example is for paper trading only.
####

# Below are the variables for development this documents
# Please do not change these variables

trade_api_url = 'https://paper-api.alpaca.markets/v2'
trade_api_wss = 'wss://paper-api.alpaca.markets/stream'
data_api_url = 'https://data.alpaca.markets/v2'
option_stream_data_wss = 'wss://data.alpaca.markets/stream'

In [16]:
# install alpaca-py
! python3 -m pip install alpaca-py

In [17]:
import json
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

import alpaca
from alpaca.trading.client import TradingClient
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.data.historical.option import OptionHistoricalDataClient
from alpaca.trading.stream import TradingStream
from alpaca.data.live.option import OptionDataStream

from alpaca.data.requests import (
    OptionBarsRequest,
    OptionTradesRequest,
    OptionLatestQuoteRequest,
    OptionLatestTradeRequest,
    OptionSnapshotRequest,
    OptionChainRequest
)
from alpaca.trading.requests import (
    GetOptionContractsRequest,
    GetAssetsRequest,
    MarketOrderRequest,
    GetOrdersRequest,
    ClosePositionRequest
)
from alpaca.trading.enums import (
    AssetStatus,
    ExerciseStyle,
    OrderSide,
    OrderType,
    TimeInForce,
    QueryOrderStatus
)
from alpaca.common.exceptions import APIError

In [18]:
# to run async code in jupyter notebook
import nest_asyncio
nest_asyncio.apply()

In [19]:
# check version of alpaca-py
alpaca.__version__

'0.34.0'

# Trading Client

In [14]:
# setup clients
trade_client = TradingClient(api_key=api_key, secret_key=secret_key, paper=paper, url_override=trade_api_url)

In [ ]:
# check trading account
# There are trhee new columns in the account object:
# - options_buying_power
# - options_approved_level
# - options_trading_level
acct = trade_client.get_account()
acct

In [ ]:
# check account configuration
# - we have new field `max_options_trading_level`
acct_config = trade_client.get_account_configurations()
acct_config

In [ ]:
# get list of assets which are options enabled
# - we can filter assets by `options_enabled` attribute
# - asset object has `options_enabled` attribute if it is options enabled
req = GetAssetsRequest(
  attributes = "options_enabled"
)
assets = trade_client.get_all_assets(req)
assets[:2]

In [ ]:
# get list of options contracts for the given underlying symbol (e.g. SPY,AAPL)
# - get_option_contracts() is a new method to get list of options contracts
# - in this example, we get 2 options contracts for SPY,AAPL
# - you can continue to fetch options contracts by specifying page_token from next_page_token of response
underlying_symbols = ["SPY", "QQQ"]
req = GetOptionContractsRequest(
    underlying_symbols = underlying_symbols,               # specify underlying symbols
    status = AssetStatus.ACTIVE,                           # specify asset status: active (default)
    expiration_date = None,                                # specify expiration date (specified date + 1 day range)
    expiration_date_gte = None,                            # we can pass date object
    expiration_date_lte = None,                            # or string (YYYY-MM-DD)
    root_symbol = None,                                    # specify root symbol
    type = None,                                           # specify option type (ContractType.CALL or ContractType.PUT)
    style = None,                                          # specify option style (ContractStyle.AMERICAN or ContractStyle.EUROPEAN)
    strike_price_gte = None,                               # specify strike price range
    strike_price_lte = None,                               # specify strike price range
    limit = 2,                                             # specify limit
    page_token = None,                                     # specify page token
)
res = trade_client.get_option_contracts(req)
res

In [ ]:
# continue to fetch option contracts if there is next_page_token in response
if res.next_page_token is not None:
    req = GetOptionContractsRequest(
        underlying_symbols = underlying_symbols,               # specify underlying symbols
        status = AssetStatus.ACTIVE,                           # specify asset status: active (default)
        expiration_date = None,                                # specify expiration date (specified date + 1 day range)
        expiration_date_gte = None,                            # we can pass date object
        expiration_date_lte = None,                            # or string (YYYY-MM-DD)
        root_symbol = None,                                    # specify root symbol
        type = None,                                           # specify option type (ContractType.CALL or ContractType.PUT)
        style = None,                                          # specify option style (ContractStyle.AMERICAN or ContractStyle.EUROPEAN)
        strike_price_gte = None,                               # specify strike price range
        strike_price_lte = None,                               # specify strike price range
        limit = 2,                                             # specify limit
        page_token = res.next_page_token,                      # specify page token
    )
    res = trade_client.get_option_contracts(req)
    display(res)

In [ ]:
# get options contract by symbol
# - get_option_contract() is a new method to get options contract by symbol or id
symbol = res.option_contracts[0].symbol
contract = trade_client.get_option_contract(symbol)
contract

In [ ]:
# get options contract by id
id = res.option_contracts[0].id
contract = trade_client.get_option_contract(symbol_or_id=id)
contract

In [ ]:
# get put options contracts
underlying_symbols = ["SPY"]

# specify expiration date range
now = datetime.now(tz = ZoneInfo("America/New_York"))
day1 = now + timedelta(days = 1)
day60 = now + timedelta(days = 60)

req = GetOptionContractsRequest(
    underlying_symbols = underlying_symbols,                     # specify underlying symbols
    status = AssetStatus.ACTIVE,                                 # specify asset status: active (default)
    expiration_date = None,                                      # specify expiration date (specified date + 1 day range)
    expiration_date_gte = day1.date(),                           # we can pass date object
    expiration_date_lte = day60.strftime(format = "%Y-%m-%d"),   # or string
    root_symbol = None,                                          # specify root symbol
    type = "put",                                                # specify option type: put
    style = ExerciseStyle.AMERICAN,                              # specify option style: american
    strike_price_gte = None,                                     # specify strike price range
    strike_price_lte = None,                                     # specify strike price range
    limit = 100,                                                 # specify limit
    page_token = None,                                           # specify page
)
res = trade_client.get_option_contracts(req)
res.option_contracts[:2]

In [ ]:
# get high open_interest contract
open_interest = 0
high_open_interest_contract = None
for contract in res.option_contracts:
    if (contract.open_interest is not None) and (int(contract.open_interest) > open_interest):
        open_interest = int(contract.open_interest)
        high_open_interest_contract = contract
high_open_interest_contract

In [ ]:
# place buy put option order
# - we can place buy put option order same as buy stock/crypto order
req = MarketOrderRequest(
    symbol = high_open_interest_contract.symbol,
    qty = 1,
    side = OrderSide.BUY,
    type = OrderType.MARKET,
    time_in_force = TimeInForce.DAY,
)
res = trade_client.submit_order(req)
res

In [ ]:
# get list of orders by specifying option contract symbol
req = GetOrdersRequest(
    status = QueryOrderStatus.ALL,
    symbols = [high_open_interest_contract.symbol],
    limit = 2,
)
orders = trade_client.get_orders(req)
orders

In [ ]:
# below cells should be done after market open otherwise there is no position for the option contract

# get positions filtered by option contract symbol
# if you do this example outside of market hours, you will see empty list
# because we have no position in this option contract
# please wait market open and run this example again
positions = trade_client.get_all_positions()
[pos for pos in positions if pos.symbol == high_open_interest_contract.symbol]

In [ ]:
# get positions by symbol
trade_client.get_open_position(symbol_or_asset_id=high_open_interest_contract.symbol)


In [ ]:
# get positions by contract id
trade_client.get_open_position(symbol_or_asset_id = high_open_interest_contract.id)

In [ ]:
# close the option position
trade_client.close_position(
    symbol_or_asset_id = high_open_interest_contract.symbol,
    close_options = ClosePositionRequest(qty = "1")
)

In [ ]:
# exercise the options position
# - this method does not return anything
trade_client.exercise_options_position(
    symbol_or_contract_id = high_open_interest_contract.symbol
)

# Trade Update (Stream)

With TradingStream client, you can get updates about trades

fyi. you can open this notebook in another window and run below cell to check trade updates.

In [ ]:
# subscribe trade updates
trade_stream_client = TradingStream(api_key, secret_key, paper=paper, url_override = trade_api_wss)

async def trade_updates_handler(data):
    print(data)

trade_stream_client.subscribe_trade_updates(trade_updates_handler)
trade_stream_client.run()

# Market Data (Historical)

In [ ]:
# setup option historical data client
option_historical_data_client = OptionHistoricalDataClient(api_key, secret_key, url_override = data_api_url)

In [ ]:
# get options historical bars by symbol
req = OptionBarsRequest(
    symbol_or_symbols = high_open_interest_contract.symbol,
    timeframe = TimeFrame(amount = 1, unit = TimeFrameUnit.Hour),   # specify timeframe
    start = now - timedelta(days = 5),                              # specify start datetime, default=the beginning of the current day.
    # end_date=None,                                                # specify end datetime, default=now
    limit = 2,                                                      # specify limit
)
option_historical_data_client.get_option_bars(req).df

In [ ]:
# get options historical trades by symbol
req = OptionTradesRequest(
    symbol_or_symbols = high_open_interest_contract.symbol,
    start = now - timedelta(days = 5),                              # specify start datetime, default=the beginning of the current day.
    # end=None,                                                     # specify end datetime, default=now
    limit = 2,                                                      # specify limit
)
option_historical_data_client.get_option_trades(req).df

In [ ]:
# get options exchange codes
option_historical_data_client.get_option_exchange_codes()

In [ ]:
# get option latest quote by symbol
req = OptionLatestQuoteRequest(
    symbol_or_symbols = [high_open_interest_contract.symbol],
)
option_historical_data_client.get_option_latest_quote(req)

In [ ]:
# get option latest trade by symbol
req = OptionLatestTradeRequest(
    symbol_or_symbols = [high_open_interest_contract.symbol],
)
option_historical_data_client.get_option_latest_trade(req)

In [ ]:
# get option snapshot by symbol
req = OptionSnapshotRequest(
    symbol_or_symbols = [high_open_interest_contract.symbol],
)
option_historical_data_client.get_option_snapshot(req)

In [ ]:
# get option chain by underlying_symbol
req = OptionChainRequest(
    underlying_symbol = high_open_interest_contract.underlying_symbol,
)
option_historical_data_client.get_option_chain(req)

# Market Data (Stream)

In [ ]:
option_data_stream_client = OptionDataStream(api_key, secret_key, url_override = option_stream_data_wss)

async def option_data_stream_handler(data):
    print(data)

symbols = [
    high_open_interest_contract.symbol,
]

option_data_stream_client.subscribe_quotes(option_data_stream_handler, *symbols)
option_data_stream_client.subscribe_trades(option_data_stream_handler, *symbols)

option_data_stream_client.run()